In [4]:
import pandas as pd
import ipywidgets as widgets
from collections import defaultdict
import collections
from IPython.display import display
from IPython.display import clear_output

In [5]:
raw_data_df = pd.read_csv('..\\download_files\\raw-data-table.csv')

In [6]:
raw_data_df.head()

,filename,archiveType,inferredVariableType,units,proxyObservationType,interpretation/variable,interpretation/variableDetail
0,A7.Oppo.2005.lpd,marine sediment,Age,yr BP,NaN,Age,calendar
1,A7.Oppo.2005.lpd,marine sediment,Age,yr BP,NaN,Age,calendar
2,A7.Oppo.2005.lpd,marine sediment,NaN,per mil,D18O,d18Osw,sea surface
3,A7.Oppo.2005.lpd,marine sediment,NaN,mmol/mol,Mg/Ca,Temperature,sea surface
4,A7.Oppo.2005.lpd,marine sediment,Sea Surface Temperature,deg C,NaN,Temperature,sea surface


In [7]:
archive_inferredVarType_df = raw_data_df.filter(['archiveType','inferredVariableType'], axis=1)

In [8]:
archive_inferredVarType_df.head()

,archiveType,inferredVariableType
0,marine sediment,Age
1,marine sediment,Age
2,marine sediment,NaN
3,marine sediment,NaN
4,marine sediment,Sea Surface Temperature


In [9]:
archive_inferredVarType_df = archive_inferredVarType_df.dropna()

In [10]:
# CALCULATING FREQUENCY FOR EACH INFERRED VARIABLE TYPE
def calc_freq(dataframe_obj, col1, col2):
    counter_dict = {}
    for index, row in dataframe_obj.iterrows():
    #     print(row['c1'], row['c2'])
        if row[col1] not in counter_dict:
            counter_dict[row[col1]] = {}
            counter_dict[row[col1]][row[col2]] = 1
            counter_dict[row[col1]]['summation'] = 1
        else:
            if row[col2] not in counter_dict[row[col1]]:
                counter_dict[row[col1]][row[col2]] = 1
                counter_dict[row[col1]]['summation'] += 1
            else:
                counter_dict[row[col1]][row[col2]] += 1
                counter_dict[row[col1]]['summation'] += 1
    return counter_dict

counter_dict_archived_inf_vartype = calc_freq(archive_inferredVarType_df, 'archiveType', 'inferredVariableType')
counter_dict_archived_inf_vartype

{'marine sediment': {'Age': 56,
  'summation': 255,
  'Sea Surface Temperature': 70,
  'Temperature': 41,
  'Year': 42,
  'D18O': 27,
  'Relative Sea Level': 2,
  'Sedimentation Rate': 3,
  'Thermocline Temperature': 4,
  'Sea Surface Salinity': 2,
  'Carbonate Ion Concentration': 4,
  'Radiocarbon Age': 1,
  'Subsurface Temperature': 1,
  'Salinity': 1,
  'Accumulation rate': 1},
 'physical sample': {'Year': 8, 'summation': 9, 'Temperature': 1},
 'lake sediment': {'Year': 35,
  'summation': 84,
  'Temperature': 26,
  'Age': 14,
  'Uncertainty temperature': 3,
  'Temperature1': 3,
  'Temperature2': 1,
  'Temperature3': 1,
  'Uncertainty temperature1': 1},
 'glacier ice': {'Year': 46, 'summation': 52, 'Temperature': 4, 'Age': 2},
 'tree': {'Year': 387, 'summation': 394, 'Temperature': 6, 'Temperature1': 1},
 'documents': {'Temperature': 15, 'summation': 30, 'Year': 15},
 'hybrid': {'Year': 1, 'summation': 2, 'Temperature': 1},
 'coral': {'SampleData': 2, 'summation': 87, 'Year': 84, 'Te

***Pre-processing***
1. Temperature 1, 2, 3?
2. Uncertainity Temperature and Uncertainity Temperature1 ?
3. D180 and D18o (units are different)

In [11]:
# CALCULATING PROBABILITY FOR EACH INFERRED VARIABLE TYPE
import copy


def calc_probability(counter_obj):
    counter_dict_copy = copy.deepcopy(counter_obj)
    for key, value in counter_dict_copy.items():
        for inner_key, inner_value in value.items():
            if inner_key != 'summation':
                value[inner_key] = inner_value/value['summation']
    return counter_dict_copy

counter_dict_archived_inf_vartype_prob = calc_probability(counter_dict_archived_inf_vartype)
counter_dict_archived_inf_vartype_prob

{'marine sediment': {'Age': 0.2196078431372549,
  'summation': 255,
  'Sea Surface Temperature': 0.27450980392156865,
  'Temperature': 0.1607843137254902,
  'Year': 0.16470588235294117,
  'D18O': 0.10588235294117647,
  'Relative Sea Level': 0.00784313725490196,
  'Sedimentation Rate': 0.011764705882352941,
  'Thermocline Temperature': 0.01568627450980392,
  'Sea Surface Salinity': 0.00784313725490196,
  'Carbonate Ion Concentration': 0.01568627450980392,
  'Radiocarbon Age': 0.00392156862745098,
  'Subsurface Temperature': 0.00392156862745098,
  'Salinity': 0.00392156862745098,
  'Accumulation rate': 0.00392156862745098},
 'physical sample': {'Year': 0.8888888888888888,
  'summation': 9,
  'Temperature': 0.1111111111111111},
 'lake sediment': {'Year': 0.4166666666666667,
  'summation': 84,
  'Temperature': 0.30952380952380953,
  'Age': 0.16666666666666666,
  'Uncertainty temperature': 0.03571428571428571,
  'Temperature1': 0.03571428571428571,
  'Temperature2': 0.011904761904761904,
  

In [12]:
inferredVarType_units_df = raw_data_df.filter(['inferredVariableType','units'], axis=1)
inferredVarType_units_df.head()

,inferredVariableType,units
0,Age,yr BP
1,Age,yr BP
2,NaN,per mil
3,NaN,mmol/mol
4,Sea Surface Temperature,deg C


In [13]:
inferredVarType_units_df = inferredVarType_units_df.dropna()
inferredVarType_units_df.head()

,inferredVariableType,units
0,Age,yr BP
1,Age,yr BP
4,Sea Surface Temperature,deg C
6,Year,AD
10,Year,AD


In [14]:
# CALCULATING FREQUENCY FOR EACH INFERRED VARIABLE TYPE AND UNITS
counter_dict_inf_var_units = calc_freq(inferredVarType_units_df, 'inferredVariableType', 'units')
counter_dict_inf_var_units

{'Age': {'yr BP': 14,
  'summation': 78,
  'BP': 16,
  'kyr BP': 39,
  'ka BP': 1,
  'cal. BP': 1,
  'year': 3,
  'kaBP': 1,
  'yr. BP': 1,
  'ky': 1,
  'yr B.P.': 1},
 'Sea Surface Temperature': {'deg C': 69, 'summation': 71, 'degC': 2},
 'Year': {'AD': 621, 'summation': 623, 'years': 1, 'year C.E.': 1},
 'Temperature': {'degC': 50, 'summation': 51, 'deg C': 1},
 'Uncertainty temperature': {'degC': 3, 'summation': 3},
 'Temperature1': {'degC': 4, 'summation': 4},
 'D18O': {'per mil': 24,
  'summation': 28,
  'per mil SMOW': 3,
  'per mil VPDB': 1},
 'SampleData': {'permil': 2, 'summation': 2},
 'Relative Sea Level': {'m': 2, 'summation': 2},
 'Sedimentation Rate': {'cm/kyr': 2, 'summation': 3, 'cm': 1},
 'Thermocline Temperature': {'deg C': 4, 'summation': 4},
 'Sea Surface Salinity': {'psu': 2, 'summation': 2},
 'Carbonate Ion Concentration': {'umol/mol': 4, 'summation': 4},
 'Radiocarbon Age': {'kyr BP': 1, 'summation': 1},
 'Subsurface Temperature': {'deg C': 1, 'summation': 1},
 '

In [15]:
# CALCULATING PROBABILITY FOR INFERRED VARIABLE TYPE UNITS
counter_dict_inf_var_units_prob = calc_probability(counter_dict_inf_var_units)
counter_dict_inf_var_units_prob

{'Age': {'yr BP': 0.1794871794871795,
  'summation': 78,
  'BP': 0.20512820512820512,
  'kyr BP': 0.5,
  'ka BP': 0.01282051282051282,
  'cal. BP': 0.01282051282051282,
  'year': 0.038461538461538464,
  'kaBP': 0.01282051282051282,
  'yr. BP': 0.01282051282051282,
  'ky': 0.01282051282051282,
  'yr B.P.': 0.01282051282051282},
 'Sea Surface Temperature': {'deg C': 0.971830985915493,
  'summation': 71,
  'degC': 0.028169014084507043},
 'Year': {'AD': 0.9967897271268058,
  'summation': 623,
  'years': 0.0016051364365971107,
  'year C.E.': 0.0016051364365971107},
 'Temperature': {'degC': 0.9803921568627451,
  'summation': 51,
  'deg C': 0.0196078431372549},
 'Uncertainty temperature': {'degC': 1.0, 'summation': 3},
 'Temperature1': {'degC': 1.0, 'summation': 4},
 'D18O': {'per mil': 0.8571428571428571,
  'summation': 28,
  'per mil SMOW': 0.10714285714285714,
  'per mil VPDB': 0.03571428571428571},
 'SampleData': {'permil': 1.0, 'summation': 2},
 'Relative Sea Level': {'m': 1.0, 'summatio

***Pre-processing***
1. Age    
    1. Before Present (BP) years is a time scale used mainly in archaeology, geology, and other scientific disciplines to specify when events occurred prior to the origin of practical radiocarbon dating in the 1950s.
    2. The cal prefix indicates that the dates are the result of radiocarbon calibration using tree ring data. These values should correspond exactly to normal historical years BC and AD. The term cal BP means the number of years before 1950 and can be directly compared to calendar years.
    3. kyr, ky = kilo year, ka - kilo annum
2. Sea Surface Temp - remove space between deg C and degC

In [16]:
archive_proxyObsType_df = raw_data_df.filter(['archiveType','proxyObservationType'], axis=1)
archive_proxyObsType_df.head()

,archiveType,proxyObservationType
0,marine sediment,NaN
1,marine sediment,NaN
2,marine sediment,D18O
3,marine sediment,Mg/Ca
4,marine sediment,NaN


In [17]:
archive_proxyObsType_df = archive_proxyObsType_df.dropna()
archive_proxyObsType_df.head()

,archiveType,proxyObservationType
2,marine sediment,D18O
3,marine sediment,Mg/Ca
5,physical sample,D18O
7,physical sample,SE
8,physical sample,D18O


In [18]:
# CALCULATING FREQ OF EACH PROXY OBSERVATION TYPE BASED ON ARCHIVE TYPE
counter_dict_archive_proxyObsType = calc_freq(archive_proxyObsType_df, 'archiveType', 'proxyObservationType')
counter_dict_archive_proxyObsType

{'marine sediment': {'D18O': 57,
  'summation': 191,
  'Mg/Ca': 36,
  'GhostMeasured': 9,
  'Depth top': 1,
  'Depth bottom': 1,
  'AgeMax': 1,
  'AgeMin': 1,
  'Uk37Prime': 18,
  'DiffuseSpectralReflectance': 1,
  'TEX86': 4,
  'BIT': 1,
  'Uk37': 7,
  'D13C': 9,
  'B/Ca': 4,
  'NumberOfObservations': 2,
  'Ba/Ca': 6,
  'AlkenoneAbundance': 2,
  'Density': 1,
  'TotalOrganicCarbon': 1,
  'N': 1,
  'C/N': 1,
  'CaCO3': 1,
  '37:3AlkenoneConcentration': 1,
  '37:2AlkenoneConcentration': 1,
  'AlkenoneConcentration': 1,
  'Al/Ca': 2,
  'Mn/Ca': 2,
  'Zn/Ca': 2,
  'Sr/Ca': 2,
  'Notes': 2,
  'Notes1': 1,
  'Mg Ca': 7,
  'Core': 3,
  'Dust': 1,
  'DryBulkDensity': 1},
 'physical sample': {'D18O': 2,
  'summation': 31,
  'SE': 1,
  'Trsgi': 4,
  'TRW': 12,
  'GhostMeasured': 1,
  'Sr Ca': 1,
  'Sr/Ca': 2,
  'Trsgi1': 2,
  'Trsgi2': 2,
  'Trsgi3': 2,
  'Trsgi4': 2},
 'lake sediment': {'Core': 2,
  'summation': 70,
  'TEX86': 4,
  'Upper95': 1,
  'Lower95': 1,
  'GhostMeasured': 13,
  'Thickn

***Pre-processing***
1. D180, D18o, D1801
2. Mg/Ca and Mg Ca
3. Notes and Notes1
4. C/N and C N
5. Cl and Chloride
6. Trsgi1,2,3,4
7. Sr/Ca and Sr Ca
8. Calcification1,2,3,4,5,6,7
9. What is ghostmeasured?


In [19]:
# CALCULATING PROBABILITY OF EACH PROXY OBSERVATION TYPE
counter_dict_archive_proxyObsType_prob = calc_probability(counter_dict_archive_proxyObsType)
counter_dict_archive_proxyObsType_prob

{'marine sediment': {'D18O': 0.29842931937172773,
  'summation': 191,
  'Mg/Ca': 0.18848167539267016,
  'GhostMeasured': 0.04712041884816754,
  'Depth top': 0.005235602094240838,
  'Depth bottom': 0.005235602094240838,
  'AgeMax': 0.005235602094240838,
  'AgeMin': 0.005235602094240838,
  'Uk37Prime': 0.09424083769633508,
  'DiffuseSpectralReflectance': 0.005235602094240838,
  'TEX86': 0.020942408376963352,
  'BIT': 0.005235602094240838,
  'Uk37': 0.03664921465968586,
  'D13C': 0.04712041884816754,
  'B/Ca': 0.020942408376963352,
  'NumberOfObservations': 0.010471204188481676,
  'Ba/Ca': 0.031413612565445025,
  'AlkenoneAbundance': 0.010471204188481676,
  'Density': 0.005235602094240838,
  'TotalOrganicCarbon': 0.005235602094240838,
  'N': 0.005235602094240838,
  'C/N': 0.005235602094240838,
  'CaCO3': 0.005235602094240838,
  '37:3AlkenoneConcentration': 0.005235602094240838,
  '37:2AlkenoneConcentration': 0.005235602094240838,
  'AlkenoneConcentration': 0.005235602094240838,
  'Al/Ca':

In [20]:
proxyObsType_units_df = raw_data_df.filter(['proxyObservationType','units'], axis=1)
proxyObsType_units_df.head()

,proxyObservationType,units
0,NaN,yr BP
1,NaN,yr BP
2,D18O,per mil
3,Mg/Ca,mmol/mol
4,NaN,deg C


In [21]:
proxyObsType_units_df = proxyObsType_units_df.dropna()
proxyObsType_units_df

,proxyObservationType,units
2,D18O,per mil
3,Mg/Ca,mmol/mol
5,D18O,permil
7,SE,degC
8,D18O,degC
...,...,...
3489,Mg/Ca,mmol/mol
3490,D18O,per mil VPDB
3494,D18O,per mil
3495,D18O,per mil


In [22]:
# CALCULATING FREQUENCY FOR EACH PROXY OBSERVATION TYPE AND UNITS
counter_dict_proxy_obs_units = calc_freq(proxyObsType_units_df, 'proxyObservationType', 'units')
counter_dict_proxy_obs_units

{'D18O': {'per mil': 46,
  'summation': 168,
  'permil': 103,
  'degC': 1,
  'percent PCD': 1,
  'percent PDB': 1,
  'per mil VPDB': 9,
  'per mil PDB': 5,
  'per mil VPBD': 1,
  'per mil SMOW': 1},
 'Mg/Ca': {'mmol/mol': 35, 'summation': 37, 'mmolmol': 1, 'Mg/Ca': 1},
 'SE': {'degC': 1, 'summation': 1},
 'TEX86': {'degC': 3, 'summation': 3},
 'Upper95': {'degC': 1, 'summation': 1},
 'Lower95': {'degC': 1, 'summation': 1},
 'DD': {'permil': 7, 'summation': 7},
 'Year old': {'AD': 1, 'summation': 1},
 'Thickness': {'mwe': 2, 'summation': 9, 'cm': 3, 'mm': 4},
 'Na': {'ng/g': 1, 'summation': 2, 'ppb': 1},
 'VarveThickness': {'cm': 2, 'summation': 5, 'mm': 3},
 'Duration': {'AD': 1, 'summation': 1},
 'BSi': {'mg/g': 1, 'summation': 3, 'percent': 1, 'mg/cm2/yr': 1},
 'X radiograph dark layer': {'dark_sum': 1, 'summation': 1},
 'VarveProperty': {'dark_sum': 1, 'summation': 1},
 'D18O1': {'permil': 1, 'summation': 1},
 'Massacum': {'g.cm-2.a-1': 1, 'summation': 1},
 'SedAccumulation': {'g.cm

In [23]:
# CALCULATING PROBABILITY OF EACH PROXY OBSERVATION TYPE
counter_dict_proxy_obs_units_prob = calc_probability(counter_dict_proxy_obs_units)
counter_dict_proxy_obs_units_prob

{'D18O': {'per mil': 0.27380952380952384,
  'summation': 168,
  'permil': 0.6130952380952381,
  'degC': 0.005952380952380952,
  'percent PCD': 0.005952380952380952,
  'percent PDB': 0.005952380952380952,
  'per mil VPDB': 0.05357142857142857,
  'per mil PDB': 0.02976190476190476,
  'per mil VPBD': 0.005952380952380952,
  'per mil SMOW': 0.005952380952380952},
 'Mg/Ca': {'mmol/mol': 0.9459459459459459,
  'summation': 37,
  'mmolmol': 0.02702702702702703,
  'Mg/Ca': 0.02702702702702703},
 'SE': {'degC': 1.0, 'summation': 1},
 'TEX86': {'degC': 1.0, 'summation': 3},
 'Upper95': {'degC': 1.0, 'summation': 1},
 'Lower95': {'degC': 1.0, 'summation': 1},
 'DD': {'permil': 1.0, 'summation': 7},
 'Year old': {'AD': 1.0, 'summation': 1},
 'Thickness': {'mwe': 0.2222222222222222,
  'summation': 9,
  'cm': 0.3333333333333333,
  'mm': 0.4444444444444444},
 'Na': {'ng/g': 0.5, 'summation': 2, 'ppb': 0.5},
 'VarveThickness': {'cm': 0.4, 'summation': 5, 'mm': 0.6},
 'Duration': {'AD': 1.0, 'summation'

In [24]:
def generate_options(dict_obj):
    sorted_list = sorted(dict_obj.items(), key=lambda x:x[1], reverse=True)
    ret_sorted_list = ['Select']
    for k,v in sorted_list:
        if k != 'summation':
            option_val = k + ' (' + str(round(v, 3)) + ')'
            ret_sorted_list.append(option_val)
    return ret_sorted_list

def display_archive_types(archive_types_list):
    archive_types_list.sort()
    archive_types_list.insert(0,'Select')
    return archive_types_list

In [25]:
style = {'description_width': 'initial', 'select_width' : 'initial'}

In [28]:
from ipywidgets import HBox, VBox

In [29]:
archive_type_dropdown = widgets.Dropdown(options = display_archive_types(list(counter_dict_archived_inf_vartype.keys())), description='Archive Type')
output_inf = widgets.Output()
output_proxy = widgets.Output()
output_inf_units = widgets.Output()
output_proxy_units = widgets.Output()

inferred_var_type_dropdown = widgets.Dropdown(options = [], description='Inferred Variable Type', style=style)
proxy_obs_type_dropdown = widgets.Dropdown(options = [], description='Proxy Observation Type', style=style)
units_inferred_var_type_dropdown = widgets.Dropdown(options =[], description='Units(Inferred Variable Type)', style=style)
units_proxy_obs_type_dropdown = widgets.Dropdown(options =[], description='Units(Proxy Observation Type)', style=style)
    
def inferred_var_type_dropdown_eventhandler(change):
    output_inf_units.clear_output()
    inferred_var_type_value = change.new
    if ' ' in inferred_var_type_value:
        inferred_var_type_value = inferred_var_type_value[:(inferred_var_type_value.rindex(' '))]
    displayChildren(counter_dict_inf_var_units_prob, inferred_var_type_value, units_inferred_var_type_dropdown, None, False)

    with output_inf_units:
        display(units_inferred_var_type_dropdown)

def proxy_obs_type_dropdown_eventhandler(change):
    output_proxy_units.clear_output()
    
    proxy_obs_type_value = change.new
    if ' ' in proxy_obs_type_value:
        proxy_obs_type_value = proxy_obs_type_value[:(proxy_obs_type_value.rindex(' '))]
    displayChildren(counter_dict_proxy_obs_units_prob, proxy_obs_type_value, units_proxy_obs_type_dropdown, None, False)
    with output_proxy_units:
        display(units_proxy_obs_type_dropdown)
    
def displayChildren(probability_dict, parent_value, dropdown_obj, dropdown_eventhandler, to_observe):
    
    if(probability_dict.get(parent_value)):
        options_list = generate_options(probability_dict.get(parent_value))
    else:
        options_list = ['Unitless']
    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')
    
def archive_type_dropdown_eventhandler(change):
    archive_type_value = change.new
    output_inf.clear_output()
    output_inf_units.clear_output()
    displayChildren(counter_dict_archived_inf_vartype_prob, archive_type_value, inferred_var_type_dropdown, inferred_var_type_dropdown_eventhandler, True)    
    with output_inf:
        display(inferred_var_type_dropdown)
        
    output_proxy.clear_output()
    output_proxy_units.clear_output()
    displayChildren(counter_dict_archive_proxyObsType_prob, archive_type_value, proxy_obs_type_dropdown, proxy_obs_type_dropdown_eventhandler, True)
    with output_proxy:
        display(proxy_obs_type_dropdown)
    
archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

display(archive_type_dropdown)


Dropdown(description='Archive Type', options=('Select', 'bivalve', 'coral', 'documents', 'glacier ice', 'h', '…

In [30]:
first_line = HBox([output_inf, output_inf_units])
second_line = HBox([output_proxy, output_proxy_units])
VBox([first_line, second_line])

In [26]:
abc = widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Ringo'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)
display(abc)

Combobox(value='', description='Combobox:', ensure_option=True, options=('Paul', 'John', 'George', 'Ringo'), p…